# runtime.net_process

> TODO fill in description

In [ ]:
#| default_exp runtime.net_process

In [ ]:
#| hide
from nbdev.showdoc import *; 

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#|export
from __future__ import annotations
import asyncio
from abc import ABC, abstractmethod
from types import MappingProxyType
from typing import Type, Tuple, Dict

import fbdev
from fbdev.exceptions import NodeError, EdgeError
from fbdev.comp.packet import Packet
from fbdev.comp.port import PortType, PortSpec, PortSpecCollection, PortID, PortCollection
from fbdev.comp.base_component import BaseComponent
from fbdev.graph.graph_spec import GraphSpec, NodeSpec
from fbdev.graph.packet_registry import TrackedPacket
from fbdev.graph.net import Edge, Node, Net
from fbdev.graph.graph_component import GraphComponentFactory
from fbdev.runtime.base_net_runtime import BaseNetRuntime

In [ ]:
#|hide
from fbdev.complib import ExecComponent
from fbdev.complib.func_component_factory import func_component

In [ ]:
#|hide
show_doc(fbdev.runtime.net_process.NetProcess)

---

### NetProcess

>      NetProcess (net:Net, stop_port:PortID=None)

*Helper class that provides a standard way to create an ABC using
inheritance.*

In [ ]:
#|export
class NetProcess(BaseNetRuntime):
    def __init__(self, net:Net, stop_port:PortID=None):
        super().__init__()
        self._net:Net = net
        self._started = False
        self._stopped = False
        self._stop_port = stop_port
        self._stop_event = asyncio.Event()
        
        if self._stop_port is not None and self._stop_port not in self._net.ports:
            raise ValueError(f"Port {self._stop_port} does not exist in net.")

    @property
    def ports(self) -> PortCollection: return self._net.ports

    def execute(self):
        """Note: this method cannot be run from within an event loop."""
        raise NotImplementedError("NetProcess does not support synchronous execution.")
    
    async def aexecute(self):
        if self._started: raise RuntimeError("NetProcess has already been started.")
        if self._stopped: raise RuntimeError("NetProcess has been terminated.")
        
        if self._stop_port is not None:
            async def stop_packet_receiver():
                packet = await self._net.ports[self._stop_port].get()
                if not self._stopped:
                    await self.stop()
            asyncio.create_task(stop_packet_receiver())
        
        await self._net.start()
        self._started = True
    
    async def await_stop(self):
        await self._stop_event.wait()
    
    async def stop(self):
        if not self._started: raise RuntimeError("NetProcess has not yet been started.")
        if self._stopped: raise RuntimeError("NetProcess has already been terminated.")
        await self._net.terminate()
        self._stopped = True
        self._stop_event.set()

In [ ]:
@func_component(loop_execution=True)
def SquareComponent(inp):
    return inp**2

async with NetProcess.from_component(SquareComponent) as ex:
    await ex.aexecute()
    
    await ex.ports.input.inp.put_value(2)
    print("2^2 =",await ex.ports.output.out.get_and_consume())
    
    await ex.ports.input.inp.put_value(5)
    print("5^2 =",await ex.ports.output.out.get_and_consume())
    
    await ex.ports.input.inp.put_value(7)
    print("7^2 =",await ex.ports.output.out.get_and_consume())


2^2 = 4
5^2 = 25
7^2 = 49


In [ ]:
class FooComponent(ExecComponent):
    port_specs = PortSpecCollection(
        PortSpec(PortType.MESSAGE, 'stop'),
    )
    async def _execute(self):
        self.send_message('stop')